## Task 2

Transformer Encoder

In [1]:
import torch
from test import evaluate_test_set
import sts_data
import siamese_lstm_attention
#import sample_class
#import sample_train
import train
import test
from importlib import reload

reload(sts_data)
from sts_data import STSData

columns_mapping = {
        "sent1": "sentence_A",
        "sent2": "sentence_B",
        "label": "relatedness_score",
    }
dataset_name = "sick"
sick_data = STSData(
    dataset_name=dataset_name,
    columns_mapping=columns_mapping,
    normalize_labels=True,
    normalization_const=5.0,
)
batch_size = 16
sick_dataloaders = sick_data.get_data_loader(batch_size=batch_size)

INFO:root:loading and preprocessing data...


  0%|          | 0/3 [00:00<?, ?it/s]

INFO:root:reading and preprocessing data completed...
INFO:root:creating vocabulary...
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:torchtext.vocab.vectors:Loading vectors from .vector_cache\wiki.simple.vec.pt
INFO:root:creating vocabulary completed...


In [2]:
output_size = 1
hidden_size = 128
vocab_size = len(sick_data.vocab)
embedding_size = 300
embedding_weights = sick_data.vocab.vectors
lstm_layers = 3
learning_rate = 1e-2
fc_hidden_size = 16
max_epochs = 5
bidirectional = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
## self attention config
self_attention_config = {
    "hidden_size": 350,  ## refers to variable 'da' in the ICLR paper
    "output_size": 30,  ## refers to variable 'r' in the ICLR paper
    "penalty": 1,  ## refers to penalty coefficient term in the ICLR paper
}

In [3]:
reload(siamese_lstm_attention)
from siamese_lstm_attention import SiameseBiLSTMAttention
## init siamese lstm
siamese_lstm_attention_model = SiameseBiLSTMAttention(
    batch_size=batch_size,
    output_size=output_size,
    hidden_size=hidden_size,
    vocab_size=vocab_size,
    embedding_size=embedding_size,
    embedding_weights=embedding_weights,
    lstm_layers=lstm_layers,
    self_attention_config=self_attention_config,
    fc_hidden_size=fc_hidden_size,
    device=device,
    bidirectional=bidirectional,
)
## move model to device
siamese_lstm_attention_model.to(device)

300


C:\Users\hp\Desktop\Saarland, let's get started!\Winter Semester 21\NN\Project\nlp_project_nnti\Final\Semantic-Textual-Similarity-main\siamese_lstm_attention.py:72: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(self.W_s1.weight)
C:\Users\hp\Desktop\Saarland, let's get started!\Winter Semester 21\NN\Project\nlp_project_nnti\Final\Semantic-Textual-Similarity-main\siamese_lstm_attention.py:74: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(self.W_s2.weight)


SiameseBiLSTMAttention(
  (embeddings): Embedding(2052, 300)
  (bilstm): LSTM(300, 128, num_layers=3, bidirectional=True)
  (W_s1): Linear(in_features=256, out_features=350, bias=True)
  (W_s2): Linear(in_features=350, out_features=30, bias=True)
)

In [32]:
reload(train)

from train import train_model
import torch.optim as optim


optimizer = torch.optim.Adam(siamese_lstm_attention_model.parameters(), lr=learning_rate, betas=(0.9, 0.98))

siamese_lstm_attention = train_model(
    model=siamese_lstm_attention_model,
    optimizer=optimizer,
    dataloader=sick_dataloaders,
    data=sick_data,
    max_epochs=max_epochs,
    config_dict={
        "device": device,
        "model_name": "siamese_lstm_attention",
        "self_attention_config": self_attention_config,
    },
)
#model()

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

Running EPOCH 1
1
torch.Size([16, 15, 300])
torch.Size([16, 15, 300])
torch.Size([16, 15, 300])
torch.Size([16, 15, 300])
torch.Size([16, 15, 300])
torch.Size([16, 15, 300])
2
4

 20%|████████████████▊                                                                   | 1/5 [00:01<00:05,  1.41s/it]


torch.Size([16, 15, 300])
Finished Training
Running EPOCH 2
1
torch.Size([16, 15, 300])
torch.Size([16, 15, 300])
torch.Size([16, 15, 300])
torch.Size([16, 15, 300])
torch.Size([16, 15, 300])
torch.Size([16, 15, 300])
2
4
torch.Size([16, 15, 300])
Finished Training


 40%|█████████████████████████████████▌                                                  | 2/5 [00:02<00:04,  1.43s/it]

...

2
4
torch.Size([16, 15, 300])
Finished Training
Running EPOCH 4
1
torch.Size([16, 15, 300])
torch.Size([16, 15, 300])
torch.Size([16, 15, 300])
torch.Size([16, 15, 300])
torch.Size([16, 15, 300])
torch.Size([16, 15, 300])
2

 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [00:05<00:01,  1.43s/it]


4
torch.Size([16, 15, 300])
Finished Training
Running EPOCH 5
1
torch.Size([16, 15, 300])
torch.Size([16, 15, 300])
torch.Size([16, 15, 300])
torch.Size([16, 15, 300])
torch.Size([16, 15, 300])
torch.Size([16, 15, 300])
2
4
torch.Size([16, 15, 300])

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:07<00:00,  1.44s/it]


Finished Training


In [34]:
reload(train)
from train import train_model
import torch.optim as optim


optimizer = torch.optim.Adam(siamese_lstm_attention_model.parameters(), lr=learning_rate, betas=(0.9, 0.98))

siamese_lstm_attention = train_model(
    model=siamese_lstm_attention_model,
    optimizer=optimizer,
    dataloader=sick_dataloaders,
    data=sick_data,
    max_epochs=max_epochs,
    config_dict={
        "device": device,
        "model_name": "siamese_lstm_attention",
        "self_attention_config": self_attention_config,
    },
)

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

Running EPOCH 1
Running loss:  11.085900497436523
Training set accuracy: 0.9146108232438565
Running loss:  10.819004440307618
Training set accuracy: 0.9440521024167537
Running loss:  10.597835636138916
Training set accuracy: 0.9538679014891386
Running loss:  10.250736236572266
Training set accuracy: 0.954905416443944
Running loss:  9.343316650390625
Training set accuracy: 0.9471753811463713
Running loss:  8.8882230758667
Training set accuracy: 0.9600596314296126
Running loss:  8.587171840667725
Training set accuracy: 0.9580062122084201
Running loss:  8.340237426757813
Training set accuracy: 0.9621494533494115
Running loss:  8.253723335266113
Training set accuracy: 0.9528408274054527
Running loss:  8.218938159942628
Training set accuracy: 0.9575896007940173
Running loss:  8.162577533721924
Training set accuracy: 0.9722288442775607
Running loss:  8.145867824554443
Training set accuracy: 0.9645753037184477
Running loss:  8.138331031799316
Training set accuracy: 0.9618322422727943
Running 

INFO:root:Evaluating accuracy on dev set


Evaluating validation set ....
Validation loss: 8.128
Validation set accuracy: 0.902
Validation loss: 8.128
Validation set accuracy: 0.901
Validation loss: 8.102
Validation set accuracy: 0.927
Validation loss: 8.177
Validation set accuracy: 0.853
Validation loss: 8.199
Validation set accuracy: 0.831
Validation loss: 8.158
Validation set accuracy: 0.871
Validation loss: 8.141
Validation set accuracy: 0.888
Validation loss: 8.141
Validation set accuracy: 0.889
Validation loss: 8.118
Validation set accuracy: 0.911
Validation loss: 8.138
Validation set accuracy: 0.891
Validation loss: 8.150
Validation set accuracy: 0.879
Validation loss: 8.073
Validation set accuracy: 0.956
Validation loss: 8.102
Validation set accuracy: 0.927
Validation loss: 8.087
Validation set accuracy: 0.942
Validation loss: 8.162
Validation set accuracy: 0.868
Validation loss: 8.115
Validation set accuracy: 0.914
Validation loss: 8.178
Validation set accuracy: 0.851
Validation loss: 8.145
Validation set accuracy: 0.8

INFO:root:new model saved
INFO:root:Train loss: 8.599603652954102 - acc: 0.9594552809480134 -- Validation loss: 8.09550952911377 - acc: 0.9010445031026999
 20%|████████████████                                                                | 1/5 [19:28<1:17:52, 1168.13s/it]

...

Validation loss: 8.031
Validation set accuracy: 0.969
Finished Training
Running EPOCH 5
Running loss:  8.019743156433105
Training set accuracy: 0.9805382492952048
Running loss:  8.021355819702148
Training set accuracy: 0.9797434099018574
Running loss:  8.018772792816161
Training set accuracy: 0.9815602107904852
Running loss:  8.017148494720459
Training set accuracy: 0.9837496787309646
Running loss:  8.023497009277344
Training set accuracy: 0.9766606478951871
Running loss:  8.021070003509521
Training set accuracy: 0.9794170492328703
Running loss:  8.018533515930176
Training set accuracy: 0.9818700821138918
Running loss:  8.015843391418457
Training set accuracy: 0.9844065004959702
Running loss:  8.020774936676025
Training set accuracy: 0.9794667105190455
Running loss:  8.02405824661255
Training set accuracy: 0.9766308486461639
Running loss:  8.024078273773194
Training set accuracy: 0.9763632472604513
Running loss:  8.021599388122558
Training set accuracy: 0.9785559299401939
Running loss:

INFO:root:Evaluating accuracy on dev set


Evaluating validation set ....
Validation loss: 8.023
Validation set accuracy: 0.977
Validation loss: 8.031
Validation set accuracy: 0.969
Validation loss: 8.026
Validation set accuracy: 0.974
Validation loss: 8.027
Validation set accuracy: 0.973
Validation loss: 8.050
Validation set accuracy: 0.950
Validation loss: 8.069
Validation set accuracy: 0.931
Validation loss: 8.032
Validation set accuracy: 0.968
Validation loss: 8.028
Validation set accuracy: 0.972
Validation loss: 8.040
Validation set accuracy: 0.960
Validation loss: 8.068
Validation set accuracy: 0.932
Validation loss: 8.071
Validation set accuracy: 0.929
Validation loss: 8.034
Validation set accuracy: 0.966
Validation loss: 8.039
Validation set accuracy: 0.961
Validation loss: 8.022
Validation set accuracy: 0.978
Validation loss: 8.050
Validation set accuracy: 0.950
Validation loss: 8.050
Validation set accuracy: 0.950
Validation loss: 8.066
Validation set accuracy: 0.934
Validation loss: 8.035
Validation set accuracy: 0.9

INFO:root:Train loss: 8.020517349243164 - acc: 0.9798730089655314 -- Validation loss: 8.0545654296875 - acc: 0.9592073803767562
100%|█████████████████████████████████████████████████████████████████████████████████| 5/5 [1:15:39<00:00, 907.90s/it]

Validation loss: 8.055
Validation set accuracy: 0.946
Finished Training


In [35]:
reload(test)
evaluate_test_set(
    model=siamese_lstm_attention_model,
    data_loader=sick_dataloaders,
    config_dict={
        "device": device,
        "model_name": "siamese_lstm_attention",
        "self_attention_config": self_attention_config,
    },
)

INFO:root:Evaluating accuracy on test set
INFO:root:Evaluating accuracy on test set


Finished testing..............
Total test set accuracy: 0.671
